In [1]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)

dhcp-10-37-2-37.nlc.guest.dcccd.edu
/Users/akarshkumar0101/nca-alife/src
/Users/akarshkumar0101/miniconda3/bin/python


In [2]:
import os, sys, glob, pickle
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [144]:
import experiment_utils
import time
import subprocess

# Create Experiment

In [145]:
! rm -rf ./experiments
! mkdir -p ./experiments

In [146]:
cfgs = experiment_utils.dict_product(dict(lr=[1e-2, 1e-3], bs=[32, 64], model=['transformer', 'cnn'], save_dir='~/data/'))  
for cfg in cfgs:
    cfg['save_dir'] += f"{cfg['model']}_{cfg['lr']}_{cfg['bs']}"  
commands = experiment_utils.create_commands(cfgs, prefix='python main.py', out_file='./experiments/main.sh')
print('\n'.join(commands))
print(len(commands), 'commands')

python main.py --lr=0.01  --bs=32 --model="transformer" --save_dir="~/data/transformer_0.01_32" 
python main.py --lr=0.01  --bs=32 --model="cnn"         --save_dir="~/data/cnn_0.01_32"         
python main.py --lr=0.01  --bs=64 --model="transformer" --save_dir="~/data/transformer_0.01_64" 
python main.py --lr=0.01  --bs=64 --model="cnn"         --save_dir="~/data/cnn_0.01_64"         
python main.py --lr=0.001 --bs=32 --model="transformer" --save_dir="~/data/transformer_0.001_32"
python main.py --lr=0.001 --bs=32 --model="cnn"         --save_dir="~/data/cnn_0.001_32"        
python main.py --lr=0.001 --bs=64 --model="transformer" --save_dir="~/data/transformer_0.001_64"
python main.py --lr=0.001 --bs=64 --model="cnn"         --save_dir="~/data/cnn_0.001_64"        
8 commands


# Launch Experiment (on this node)

In [147]:
header = '\n'.join(['#!/bin/bash', 'source ~/.virtualenvs/nca-alife/bin/activate', 'cd ~/nca-alife/src'])
gpus = [4, 5, 6, 7]
# log_dir = f'~/nca-alife-data/logs/{time.strftime("%Y_%m_%d_%H_%M_%S")}'
log_dir = f'./experiments/logs/{time.strftime("%Y_%m_%d_%H_%M_%S")}'
exp_sh = f'./experiments/main.sh'
with open(exp_sh, 'r') as f:
    commands = [c for c in f.read().split('\n') if c]
print(f"{exp_sh} has {len(commands)} commands: ")
print('\n'.join(commands[:3]), '\n...')
print(f"Logging to {log_dir}")

./experiments/main.sh has 8 commands: 
python main.py --lr=0.01  --bs=32 --model="transformer" --save_dir="~/data/transformer_0.01_32" 
python main.py --lr=0.01  --bs=32 --model="cnn"         --save_dir="~/data/cnn_0.01_32"         
python main.py --lr=0.01  --bs=64 --model="transformer" --save_dir="~/data/transformer_0.01_64"  
...
Logging to ./experiments/logs/2024_06_05_11_32_45


In [148]:
!mkdir -p {log_dir}
for i_gpu, gpu in enumerate(gpus):
    job_ids = np.arange(len(commands))[i_gpu::len(gpus)]
    cvd = f'export CUDA_VISIBLE_DEVICES={gpu}'
    txt = f"{header}\n{cvd}\n\n"
    for job_id in job_ids:
        txt += f"touch {log_dir}/job_{job_id:03d}.status\n"
        txt += f"{commands[job_id]} &> {log_dir}/job_{job_id:03d}.out\n"
        txt += f"echo $? > {log_dir}/job_{job_id:03d}.status\n\n"
    with open(f"{log_dir}/gpu_{gpu}.sh", "w") as f:
        f.write(txt)

In [149]:
print('\n'.join(txt.split('\n')[:12]))

#!/bin/bash
source ~/.virtualenvs/nca-alife/bin/activate
cd ~/nca-alife/src
export CUDA_VISIBLE_DEVICES=7

touch ./experiments/logs/2024_06_05_11_32_45/job_003.status
python main.py --lr=0.01  --bs=64 --model="cnn"         --save_dir="~/data/cnn_0.01_64"          &> ./experiments/logs/2024_06_05_11_32_45/job_003.out
echo $? > ./experiments/logs/2024_06_05_11_32_45/job_003.status

touch ./experiments/logs/2024_06_05_11_32_45/job_007.status
python main.py --lr=0.001 --bs=64 --model="cnn"         --save_dir="~/data/cnn_0.001_64"         &> ./experiments/logs/2024_06_05_11_32_45/job_007.out
echo $? > ./experiments/logs/2024_06_05_11_32_45/job_007.status


In [153]:
for i_gpu, gpu in enumerate(gpus):
    command = f"sh {log_dir}/gpu_{gpu}.sh"
    print(command)
    # process = subprocess.Popen(command, shell=True)

sh ./experiments/logs/2024_06_05_11_32_45/gpu_4.sh
sh ./experiments/logs/2024_06_05_11_32_45/gpu_5.sh
sh ./experiments/logs/2024_06_05_11_32_45/gpu_6.sh
sh ./experiments/logs/2024_06_05_11_32_45/gpu_7.sh


# Track Experiment